In [20]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
data = pd.read_csv('dataMining/dstest.csv')

In [3]:
data.head()

,company,live,mediaEngine,p2p,cdn,upload,peers_count,peers_match,timestamp,sessionDuration,playbackErrorCount,totalPlaybackErrorCount,content
0,Streamroot TV,VOD,ME1,0.0,8507376.0,0.0,0.0,1,1562469807407,120000.0,0,0,content-09235
1,Streamroot TV,VOD,ME1,0.0,5778744.0,0.0,0.0,0,1562469927310,240000.0,0,0,content-09235
2,Streamroot TV,VOD,ME1,0.0,7692960.0,0.0,0.0,0,1562470047330,360000.0,0,0,content-09235
3,Streamroot TV,VOD,ME1,0.0,8229888.0,0.0,0.0,0,1562470167720,480000.0,0,0,content-09235
4,Streamroot TV,VOD,ME1,0.0,10387000.0,0.0,0.0,0,1562470287353,600000.0,0,0,content-09235


In [4]:
## Question 1
## I would define the performace as the ratio of p2p over cdn for each company
## Having a high ratio means that most of the stream is supported by the peers and that is better

def ratio_p2p_cdn (df):
    return (df.p2p.mean()/df.cdn.mean()) * 100

data.groupby('company').apply(lambda df: ratio_p2p_cdn(df))

company
Flash Infos       65.031872
Sports News       28.190941
Streamroot TV    120.891015
dtype: float64

In [5]:
### We can also use this for checking the live streaming

data.groupby(['company', 'live']).apply(lambda df: ratio_p2p_cdn(df))

company        live
Flash Infos    Live    102.378527
               VOD      45.987885
Sports News    Live     31.544646
               VOD       1.951055
Streamroot TV  VOD     120.891015
dtype: float64

In [8]:
## Question 2.1

## We want those with the smallest sessionDuration, the first two minutes

data[data.sessionDuration == data.sessionDuration.min()]

,company,live,mediaEngine,p2p,cdn,upload,peers_count,peers_match,timestamp,sessionDuration,playbackErrorCount,totalPlaybackErrorCount,content
0,Streamroot TV,VOD,ME1,0.0,8507376.0,0.0,0.00,1,1562469807407,120000.0,0,0,content-09235
16,Streamroot TV,VOD,ME1,0.0,17608456.0,0.0,0.00,1,1562477387430,120000.0,0,0,content-17013
21,Streamroot TV,VOD,ME1,166409328.0,11303312.0,0.0,19.11,38,1562468705805,120000.0,0,0,content-08963
53,Streamroot TV,VOD,ME1,0.0,14936600.0,0.0,0.00,0,1562469688171,120000.0,0,0,content-14278
74,Streamroot TV,VOD,ME1,0.0,15740112.0,0.0,0.00,0,1562478964956,120000.0,0,0,content-16536
...,...,...,...,...,...,...,...,...,...,...,...,...,...
385101,Streamroot TV,VOD,ME1,122891464.0,1946928.0,0.0,6.22,9,1562478081892,120000.0,0,0,content-01158
385107,Streamroot TV,VOD,ME1,0.0,6409296.0,0.0,0.00,0,1562465908584,120000.0,0,0,content-04063
385124,Streamroot TV,VOD,ME1,0.0,3240932.0,0.0,0.00,1,1562466022339,120000.0,0,0,content-07336
385134,Streamroot TV,VOD,ME1,0.0,19570048.0,0.0,0.00,0,1562459158348,120000.0,0,0,content-15027


In [9]:
### Question 2.2

data = data.assign(isFirstPayload = data.sessionDuration == data.sessionDuration.min())

In [10]:
### Question 2.3

data[data.isFirstPayload].groupby('company').apply(len)

company
Flash Infos        805
Sports News        115
Streamroot TV    19914
dtype: int64

In [75]:
### Question 3.1
## For the 5 sessions:
### The playbackErrorCount will be 0, 1, 0, 1, 0
### The totalPlaybackErrorCount will be 0, 1, 1, 2, 2 

In [142]:
### Question 3.2
distinct_viewers = len(data[data.isFirstPayload]) 
dv_max_duration = data.sort_values(by = 'sessionDuration',ascending=False).head(distinct_viewers) #distinct viewers in the end of session for each
dv_max_duration[dv_max_duration['totalPlaybackErrorCount'] >= 1] #filter the ones hat had at least 1 playback error

,company,live,mediaEngine,p2p,cdn,upload,peers_count,peers_match,timestamp,sessionDuration,playbackErrorCount,totalPlaybackErrorCount,content,isFirstPayload,readableDate
277333,Sports News,Live,ME2,0.0,51191953.0,58641741.0,18.98,23,1562479176789,17400000.0,0,1,content-11564,False,2019-07-07 07:57:36
277332,Sports News,Live,ME2,0.0,49831341.0,73348665.0,18.46,12,1562479056789,17280000.0,0,1,content-11564,False,2019-07-07 07:55:36
277331,Sports News,Live,ME2,5261657.0,54346911.0,61893541.0,19.52,26,1562478936821,17160000.0,0,1,content-11564,False,2019-07-07 07:53:36
277330,Sports News,Live,ME2,12343592.0,52613517.0,56071336.0,18.80,18,1562478816787,17040000.0,0,1,content-11564,False,2019-07-07 07:51:36
277329,Sports News,Live,ME2,893449.0,49953577.0,57614760.0,19.07,19,1562478696780,16920000.0,0,1,content-11564,False,2019-07-07 07:49:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272962,Sports News,Live,ME2,50929567.0,1719672.0,9143859.0,19.81,2,1562473669855,7320000.0,1,1,content-11566,False,2019-07-07 06:25:49
306126,Sports News,Live,ME2,0.0,0.0,10438127.0,19.70,6,1562474346900,7320000.0,0,1,content-11572,False,2019-07-07 06:37:06
110079,Sports News,Live,ME2,0.0,56063241.0,0.0,1.00,15,1562472152032,7320000.0,1,1,content-11564,False,2019-07-07 06:00:32
126999,Sports News,Live,ME2,0.0,51933964.0,0.0,6.00,0,1562474405428,7320000.0,1,1,content-11567,False,2019-07-07 06:38:05


In [143]:
### Question 4.1
data['readableDate'] = data['timestamp'].apply(lambda x: (datetime.fromtimestamp(x/1000) - timedelta(minutes = 2)).strftime('%Y-%m-%d %H:%M:%S'))
##if the want only this 11:33 , I can change format, but it needs the date

In [104]:
## Lets keep the data only for Streamroot TV since it is the focus for question 4
dataSR = data[data.company =='Streamroot TV']

In [144]:
selected_cont = dataSR[data.content == 'content-05335']


<ipython-input-144-04b57222d8fc>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  selected_cont = dataSR[data.content == 'content-05335']


In [ ]:
##Question 4.2
